In [2]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import time as time

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size**2)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum (np.argmax(predictions,1) == np.argmax(labels,1))
            / predictions.shape[0])

### Problem 1
L2 Regularization for logistic and NN models

In [32]:
# logistic model
train_subset = 10000
num_steps = 801
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    
    #input data
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables
    weights = tf.Variable(
        tf.truncated_normal([image_size*image_size, num_labels])
    )
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #training computation
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    # l2 regularization
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    )
    + 0.5* beta * tf.nn.l2_loss(weights)
    
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases
    )


tic = time.clock()
with tf.Session(graph = graph) as sess:
    tf.initialize_all_variables().run()
    print('initialized')
    for step in range(num_steps):
        _, loss_value, predictions = sess.run(
            [optimizer, loss, train_prediction])
    
        if(step % 100 == 0):
            print('Loss at step %d: %f' % (step, loss_value))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(
                test_prediction.eval(), test_labels))
toc = time.clock()
toc-tic

initialized
Loss at step 0: 16.616661
Training accuracy: 9.6%
Validation accuracy: 12.0%
Loss at step 100: 2.411182
Training accuracy: 71.0%
Validation accuracy: 70.0%
Loss at step 200: 1.891785
Training accuracy: 74.4%
Validation accuracy: 72.8%
Loss at step 300: 1.622716
Training accuracy: 76.0%
Validation accuracy: 74.0%
Loss at step 400: 1.445132
Training accuracy: 77.3%
Validation accuracy: 74.4%
Loss at step 500: 1.314978
Training accuracy: 78.1%
Validation accuracy: 74.7%
Loss at step 600: 1.213659
Training accuracy: 78.8%
Validation accuracy: 74.9%
Loss at step 700: 1.132009
Training accuracy: 79.2%
Validation accuracy: 75.1%
Loss at step 800: 1.064680
Training accuracy: 79.8%
Validation accuracy: 75.3%
Test accuracy: 82.3%


135.62027899999998

In [33]:
batch_size = 128
num_hidden = 1024
num_steps = 3001
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    
    # mini-batch => placeholder
    
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape = (batch_size, image_size**2))
    tf_train_labels = tf.placeholder(tf.float32,
                                    shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size**2, num_hidden]))
    biases_1 = tf.Variable(tf.zeros([num_hidden]))
    weights_2 = tf.Variable(
         tf.truncated_normal([num_hidden, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training
    logits_1 = tf.nn.relu(
        tf.matmul(tf_train_dataset, weights_1) + biases_1
    )
    logits_2 = tf.matmul(logits_1, weights_2) + biases_2
    
    # l2 regularization
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits_2, tf_train_labels
        )
    )
    + 0.5 * beta * (tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_1))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits_2)
    valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights_1) + biases_1)
            ,weights_2)
        + biases_2
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_test_dataset, weights_1) + biases_1)
            ,weights_2)
        + biases_2
    )

tic = time.clock()

with tf.Session(graph = graph) as sess:
    tf.initialize_all_variables().run()
    print('initialized')
    for step in range (num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        
        _, loss_value, predictions = sess.run(
            [optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, loss_value))
            print('Miniibatch accruacy: %.1f%%' % accuracy(
                    predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))

toc = time.clock()
toc-tic

initialized
Minibatch loss at step 0: 316.524078
Miniibatch accruacy: 12.5%
Validation accuracy: 25.1%
Minibatch loss at step 500: 8.721197
Miniibatch accruacy: 78.9%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 4.725790
Miniibatch accruacy: 81.2%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 9.551991
Miniibatch accruacy: 71.1%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 5.325856
Miniibatch accruacy: 85.2%
Validation accuracy: 80.4%
Minibatch loss at step 2500: 5.038846
Miniibatch accruacy: 79.7%
Validation accuracy: 81.9%
Minibatch loss at step 3000: 2.114545
Miniibatch accruacy: 84.4%
Validation accuracy: 80.2%
Test accuracy: 88.1%


222.15775899999994

### Problem 2

In [37]:
batch_size = 128
num_hidden = 1024
num_steps = 3001
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    
    # mini-batch => placeholder
    
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape = (batch_size, image_size**2))
    tf_train_labels = tf.placeholder(tf.float32,
                                    shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size**2, num_hidden]))
    biases_1 = tf.Variable(tf.zeros([num_hidden]))
    weights_2 = tf.Variable(
         tf.truncated_normal([num_hidden, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training
    logits_1 = tf.nn.relu(
        tf.matmul(tf_train_dataset, weights_1) + biases_1
    )
    logits_2 = tf.matmul(logits_1, weights_2) + biases_2
    
    # l2 regularization
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits_2, tf_train_labels
        )
    )
    + 0.5 * beta * (tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_1))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits_2)
    valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights_1) + biases_1)
            ,weights_2)
        + biases_2
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_test_dataset, weights_1) + biases_1)
            ,weights_2)
        + biases_2
    )

tic = time.clock()

with tf.Session(graph = graph) as sess:
    tf.initialize_all_variables().run()
    print('initialized')
    for step in range (num_steps):
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        offset = batch_size*np.random.choice(np.arange(3))
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        
        _, loss_value, predictions = sess.run(
            [optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, loss_value))
            print('Miniibatch accruacy: %.1f%%' % accuracy(
                    predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))

toc = time.clock()
toc-tic

initialized
Minibatch loss at step 0: 366.517639
Miniibatch accruacy: 7.0%
Validation accuracy: 33.9%
Minibatch loss at step 500: 0.000000
Miniibatch accruacy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 1000: 0.000000
Miniibatch accruacy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 1500: 0.000000
Miniibatch accruacy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 2000: 0.000000
Miniibatch accruacy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 2500: 0.000000
Miniibatch accruacy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 3000: 0.000000
Miniibatch accruacy: 100.0%
Validation accuracy: 74.0%
Test accuracy: 81.2%


184.65189799999985

### Problem 3

In [39]:
# without l2 regularization and overfitting

batch_size = 128
num_hidden = 1024
num_steps = 3001
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():
    
    # mini-batch => placeholder
    
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape = (batch_size, image_size**2))
    tf_train_labels = tf.placeholder(tf.float32,
                                    shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size**2, num_hidden]))
    biases_1 = tf.Variable(tf.zeros([num_hidden]))
    weights_2 = tf.Variable(
         tf.truncated_normal([num_hidden, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training
    logits_1 = tf.nn.relu(
        tf.matmul(tf_train_dataset, weights_1) + biases_1
    )
    
    # dropout
    after_dropout = tf.nn.dropout(
        logits_1, keep_prob
    )
    
    logits_2 = tf.matmul(after_dropout, weights_2) + biases_2
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits_2, tf_train_labels
        )
    )
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits_2)
    valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights_1) + biases_1)
            ,weights_2)
        + biases_2
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_test_dataset, weights_1) + biases_1)
            ,weights_2)
        + biases_2
    )

tic = time.clock()

with tf.Session(graph = graph) as sess:
    tf.initialize_all_variables().run()
    print('initialized')
    for step in range (num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        
        _, loss_value, predictions = sess.run(
            [optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, loss_value))
            print('Miniibatch accruacy: %.1f%%' % accuracy(
                    predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))

toc = time.clock()
toc-tic

initialized
Minibatch loss at step 0: 446.049164
Miniibatch accruacy: 13.3%
Validation accuracy: 36.1%
Minibatch loss at step 500: 21.559002
Miniibatch accruacy: 67.2%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 12.015055
Miniibatch accruacy: 71.9%
Validation accuracy: 80.3%
Minibatch loss at step 1500: 11.141635
Miniibatch accruacy: 71.1%
Validation accuracy: 77.2%
Minibatch loss at step 2000: 11.480363
Miniibatch accruacy: 70.3%
Validation accuracy: 79.2%
Minibatch loss at step 2500: 3.981692
Miniibatch accruacy: 79.7%
Validation accuracy: 79.7%
Minibatch loss at step 3000: 5.474179
Miniibatch accruacy: 72.7%
Validation accuracy: 79.2%
Test accuracy: 86.9%


191.05280500000003

In [40]:
# overfitting + dropout
# without l2 regularization

batch_size = 128
num_hidden = 1024
num_steps = 3001
keep_prob = 0.3

graph = tf.Graph()
with graph.as_default():
    
    # mini-batch => placeholder
    
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape = (batch_size, image_size**2))
    tf_train_labels = tf.placeholder(tf.float32,
                                    shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size**2, num_hidden]))
    biases_1 = tf.Variable(tf.zeros([num_hidden]))
    weights_2 = tf.Variable(
         tf.truncated_normal([num_hidden, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training
    logits_1 = tf.nn.relu(
        tf.matmul(tf_train_dataset, weights_1) + biases_1
    )
    
    # dropout
    after_dropout = tf.nn.dropout(
        logits_1, keep_prob
    )
    
    logits_2 = tf.matmul(after_dropout, weights_2) + biases_2
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits_2, tf_train_labels
        )
    )
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits_2)
    valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights_1) + biases_1)
            ,weights_2)
        + biases_2
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_test_dataset, weights_1) + biases_1)
            ,weights_2)
        + biases_2
    )

tic = time.clock()

with tf.Session(graph = graph) as sess:
    tf.initialize_all_variables().run()
    print('initialized')
    for step in range (num_steps):
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        offset = batch_size*np.random.choice(np.arange(3))
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        
        _, loss_value, predictions = sess.run(
            [optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, loss_value))
            print('Miniibatch accruacy: %.1f%%' % accuracy(
                    predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))

toc = time.clock()
toc-tic

initialized
Minibatch loss at step 0: 631.485229
Miniibatch accruacy: 9.4%
Validation accuracy: 39.3%
Minibatch loss at step 500: 2.806190
Miniibatch accruacy: 98.4%
Validation accuracy: 77.3%
Minibatch loss at step 1000: 0.000000
Miniibatch accruacy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 1500: 1.116743
Miniibatch accruacy: 99.2%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 0.000000
Miniibatch accruacy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 2500: 0.369271
Miniibatch accruacy: 99.2%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 0.000000
Miniibatch accruacy: 100.0%
Validation accuracy: 78.1%
Test accuracy: 84.8%


179.80294500000014

### Problem 4


In [6]:
# 4-layer (3-hidden layer)
# l2 regularization
# dropout
# learning rate decay
# witout overfitting

batch_size = 128
num_hidden_1 = 1024
num_hidden_2 = 600
num_hidden_3 = 200
hidden_sd_1 = np.sqrt(2.0/1000)
hidden_sd_2 = np.sqrt(2.0/num_hidden_1)
hidden_sd_3 = np.sqrt(2.0/num_hidden_2)
output_sd = np.sqrt(2.0/num_hidden_3)
num_steps = 20000
keep_prob_1 = 1.0
keep_prob_2 = 0.6
keep_prob_3 = 0.7
beta_1 = 0.0001
beta_2 = 0.0001
beta_3 = 0.0001
beta_4 = 0.0001

graph = tf.Graph()
with graph.as_default():
    
    # mini-batch => placeholder
    
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape = (batch_size, image_size**2))
    tf_train_labels = tf.placeholder(tf.float32,
                                    shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    hidden_weights_1 = tf.Variable(
        tf.truncated_normal([image_size**2, num_hidden_1],
                           #stddev = 0.05))
                            stddev = hidden_sd_1))
    hidden_biases_1 = tf.Variable(tf.zeros([num_hidden_1]))
    
    hidden_weights_2 = tf.Variable(
        tf.truncated_normal([num_hidden_1, num_hidden_2],
                            #stddev = 0.05))
                            stddev = hidden_sd_2))
    hidden_biases_2 = tf.Variable(tf.zeros([num_hidden_2]))
    
    hidden_weights_3 = tf.Variable(
        tf.truncated_normal([num_hidden_2, num_hidden_3],
                            #stddev = 0.05))
                            stddev = hidden_sd_3))
    hidden_biases_3 = tf.Variable(tf.zeros([num_hidden_3]))
    
    output_weights = tf.Variable(
         tf.truncated_normal([num_hidden_3, num_labels],
                            #stddev = 0.05))
                             stddev = output_sd))
    output_biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training + dropout
    hidden_layer_1 = tf.nn.relu(
        tf.matmul(tf_train_dataset, hidden_weights_1)
        + hidden_biases_1
    )
    after_dropout_1 = tf.nn.dropout(hidden_layer_1, keep_prob_1)
    
    hidden_layer_2 = tf.nn.relu(
        tf.matmul(after_dropout_1, hidden_weights_2)
        + hidden_biases_2
    )
    after_dropout_2 = tf.nn.dropout(hidden_layer_2, keep_prob_2)
    
    hidden_layer_3 = tf.nn.relu(
        tf.matmul(after_dropout_2, hidden_weights_3)
        + hidden_biases_3
    )
    after_dropout_3 = tf.nn.dropout(hidden_layer_3, keep_prob_3)
    
    output_logit = tf.matmul(after_dropout_3, output_weights) + output_biases
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            output_logit, tf_train_labels
        )
    ) + 0.5 * (beta_1 * tf.nn.l2_loss(hidden_weights_1)
               + beta_2 * tf.nn.l2_loss(hidden_weights_2)
               + beta_3 * tf.nn.l2_loss(hidden_weights_3)
               + beta_4 * tf.nn.l2_loss(output_weights))
    
    # learning rate decay
    global_step = tf.Variable(0)
    starter_learning_rate = 0.4
    learning_rate = tf.train.exponential_decay(
        starter_learning_rate, global_step, 1000, 0.96, staircase = True)
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(
        learning_rate).minimize(loss, global_step = global_step)
    
    train_prediction = tf.nn.softmax(output_logit)
        
    # validation setup
    
    valid_hidden_layer_1 = tf.nn.relu(
        tf.matmul(tf_valid_dataset, hidden_weights_1)
        + hidden_biases_1
    )
    valid_hidden_layer_2 = tf.nn.relu(
        tf.matmul(valid_hidden_layer_1, hidden_weights_2)
        + hidden_biases_2
    )
    valid_hidden_layer_3 = tf.nn.relu(
        tf.matmul(valid_hidden_layer_2, hidden_weights_3)
        + hidden_biases_3
    )
    valid_logits = tf.matmul(valid_hidden_layer_3, output_weights) + output_biases
    valid_prediction = tf.nn.softmax(valid_logits)
    
    # test setup
    
    test_hidden_layer_1 = tf.nn.relu(
        tf.matmul(tf_test_dataset, hidden_weights_1)
        + hidden_biases_1
    )
    test_hidden_layer_2 = tf.nn.relu(
        tf.matmul(test_hidden_layer_1, hidden_weights_2)
        + hidden_biases_2
    )
    test_hidden_layer_3 = tf.nn.relu(
        tf.matmul(test_hidden_layer_2, hidden_weights_3)
        + hidden_biases_3
    )
    test_logits = tf.matmul(test_hidden_layer_3, output_weights) + output_biases
    test_prediction = tf.nn.softmax(test_logits)
 
tic = time.clock()

with tf.Session(graph = graph) as sess:
    tf.initialize_all_variables().run()
    print('initialized')
    for step in range (num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        
        _, loss_value, predictions = sess.run(
            [optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, loss_value))
            print('Miniibatch accruacy: %.1f%%' % accuracy(
                    predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))

toc = time.clock()
toc-tic

initialized
Minibatch loss at step 0: 2.621596
Miniibatch accruacy: 6.2%
Validation accuracy: 40.4%
Minibatch loss at step 500: 0.616280
Miniibatch accruacy: 81.2%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 0.494740
Miniibatch accruacy: 85.9%
Validation accuracy: 86.7%
Minibatch loss at step 1500: 0.612709
Miniibatch accruacy: 82.0%
Validation accuracy: 87.2%
Minibatch loss at step 2000: 0.404161
Miniibatch accruacy: 88.3%
Validation accuracy: 87.9%
Minibatch loss at step 2500: 0.510343
Miniibatch accruacy: 84.4%
Validation accuracy: 88.7%
Minibatch loss at step 3000: 0.388806
Miniibatch accruacy: 89.1%
Validation accuracy: 88.8%
Minibatch loss at step 3500: 0.305021
Miniibatch accruacy: 91.4%
Validation accuracy: 89.1%
Minibatch loss at step 4000: 0.318781
Miniibatch accruacy: 92.2%
Validation accuracy: 89.3%
Minibatch loss at step 4500: 0.286590
Miniibatch accruacy: 92.2%
Validation accuracy: 89.4%
Minibatch loss at step 5000: 0.436625
Miniibatch accruacy: 89.8%
Validati

2141.729807